<a href="https://colab.research.google.com/github/nncliff/qwen-32B/blob/main/chapter-1/ipynb/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ハッシュベース検索による長文ドキュメントQAシミュレーション

## 目的
このノートブックでは、簡略化された**検索拡張生成（RAG）**パイプラインをデモンストレーションします。大規模なQAシステムでは、長いドキュメント全体をモデルに入力するのは非効率的です。代わりに、以下の手順を行います：
1.  ドキュメントのチャンク（段落）を**インデックス化**する。
2.  与えられたクエリに対して最も関連性の高いチャンクのみを**検索**する。
3.  クエリと取得したコンテキストを使用して回答を**生成**する。

## コード概要
このシミュレーションは3つの主要コンポーネントで構成されています：

1.  **`generate_fake_document`**:
    - ランダムな埋め込みベクトルを生成してドキュメントをシミュレートします。
    - 各「段落」は中心テーマ（`base`）周辺のベクトルクラスタです。

2.  **`HashIndex`**:
    - シンプルな検索インデックスを実装します。
    - ベクトル値に基づくハッシュ関数を使用して、類似したベクトルを同じバケットにグループ化します。
    - **検索**: クエリをハッシュして関連するバケットを見つけ、そのバケット内で正確なコサイン類似度を計算してtop-kのマッチを見つけます。これは**局所性敏感ハッシュ（LSH）**や転置ファイルインデックスを模倣しています。

3.  **`LongDocQA` モデル**:
    - 以下を含むダミーニューラルネットワーク：
        - `query_encoder`: 入力クエリをエンコードします。
        - `answer_decoder`: 取得したコンテキストを「読み取る」GRUベースのデコーダー。
        - `output_proj`: 隠れ状態を回答ベクトルに射影します。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import string
from typing import List, Dict, Tuple
import hashlib

In [ ]:
def generate_fake_document(num_paragraphs: int = 5, tokens_per_paragraph: int = 10, dim: int = 128) -> List[torch.Tensor]:
    """Generates a fake document with random sentences."""
    document = []
    for _ in range(num_paragraphs):
        base = torch.randn(dim) # shape: (dim,)

        # broadcast to create a paragraph
        paragraph = base + 0.01 * torch.randn(tokens_per_paragraph, dim) # shape: (tokens_per_paragraph, dim)

        document.append(paragraph)

    return document # List of tensors representing paragraphs of shape (tokens_per_paragraph, dim)

### テンソル変換に関する注意
以下の`_hash_vector`メソッドでは、次のコードが使用されています：
```python
h = hashlib.md5(vector[:3].detach().numpy().tobytes()).hexdigest()
```
**`.detach()`が必要な理由：**
インデックスに渡されるベクトルは、多くの場合モデル（`query_encoder`など）から来るため、`requires_grad=True`が設定されています。PyTorchでは、勾配を持つテンソルを直接NumPy配列に変換することは許可されていません。これは計算グラフを壊すためです。`.detach()`は勾配を必要としないテンソルのビューを作成します。

**`.cpu()`だけでは不十分な理由：**
GPUで実行している場合、NumPyに変換する前に`.cpu()`を呼び出してデータをホストメモリに移動する必要があります。しかし、`.cpu()`は勾配履歴を**保持**します。そのため、テンソルが勾配を必要とする場合、`tensor.cpu().numpy()`は依然として失敗します。`.detach()`を使用する必要があり（GPUの場合は`.cpu()`も併用）、`tensor.detach().cpu().numpy()`のようにします。

In [ ]:
class HashIndex:
    def __init__(self, key_vectors: List[torch.Tensor], num_buckets: int = 16):
        self.buckets: Dict[int, List[Tuple[int, torch.Tensor]]] = {i: [] for i in range(num_buckets)}
        self.num_buckets = num_buckets

        for idx, paragraph in enumerate(key_vectors):
            # Hash is based on the mean vector of the paragraph
            key = paragraph.mean(dim=0)  # the shape of key: (dim,)

            bucket_id = self._hash_vector(key) # shape: ()
            self.buckets[bucket_id].append((idx, key))

    def _hash_vector(self, vector: torch.Tensor) -> int:
        """Hashes a vector to a bucket ID."""
        h = hashlib.md5(vector[:3].detach().numpy().tobytes()).hexdigest()  # Use first 3 elements for hashing
        return int(h, 16) % self.num_buckets

    def search(self, query_vector: torch.Tensor, top_k: int = 3) -> List[int]:
        """Searches for the top_k closest key vectors to the query_vector."""
        bucket_id = self._hash_vector(query_vector)
        candidates = self.buckets[bucket_id]
        scores = []

        for idx, vector in candidates:
            # cosine similarity is expecting 2D tensors (batch_size, dim)
            # .item() to get scalar value from tensor
            dist = F.cosine_similarity(query_vector.unsqueeze(0), vector.unsqueeze(0)).item()
            scores.append((idx, dist))

        # Get top_k closest
        scores.sort(key=lambda x: x[1])  # Sort by similarity
        return [idx for idx, _ in scores[:top_k]]

In [ ]:
class LongDocQA(nn.Module):
    def __init__(self, dim: int = 128):
        super(LongDocQA, self).__init__()

        self.query_encoder = nn.Sequential(
            nn.Linear(dim, dim),
            nn.Tanh()
        )

        self.answer_decoder = nn.GRU(input_size=dim, hidden_size=dim, batch_first=True)
        self.output_proj = nn.Linear(dim, dim)

    def forward(self, query_vector: torch.Tensor, context_vectors: List[torch.Tensor]) -> torch.Tensor:
        input_seq = torch.stack(context_vectors, dim=0).unsqueeze(0)  # shape: (1, num_contexts, dim)
        _, hidden = self.answer_decoder(input_seq)  # shape: (1, 1, dim)
        response = self.output_proj(hidden.squeeze(0))  # shape: (1, dim)
        return response

In [ ]:
# Construct fake document and index
torch.manual_seed(42)
dim = 64
doc = generate_fake_document(num_paragraphs=30, tokens_per_paragraph=8, dim=dim)
#index = HashIndex(doc_summary_vector, num_buckets=8)
index = HashIndex(doc, num_buckets=8)

# Simulate multiple turns of user queries
model = LongDocQA(dim=dim)
model.eval()

for i in range(1, 6):
    # Simulate a user query (main topic and paragraph are relevant)
    base_para = random.choice(doc)
    query_vector = base_para.mean(dim=0) + 0.02 * torch.randn(dim)  # shape: (dim,)
    query_encoded = model.query_encoder(query_vector)  # shape: (dim,)

    # Search for relevant paragraphs
    top_indices = index.search(query_encoded, top_k=3)
    selected_paragraphs = [doc[idx].mean(dim=0) for idx in top_indices]  # List of tensors of shape (dim,)

    # Generate answer
    with torch.no_grad():
        answer_vector = model(query_encoded, selected_paragraphs)  # shape: (dim,)

    keywords = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
    print(f"Turn {i}: Top paragraphs indices: {top_indices}")
    print(f"Generate answer summary (norm): {answer_vector.norm().item():.4f} with Answer: {keywords}\n")

Turn 1: Top paragraphs indices: [25, 4, 14]
Generate answer summary (norm): 1.6464 with Answer: X26Y9W

Turn 2: Top paragraphs indices: [1, 20]
Generate answer summary (norm): 1.3438 with Answer: SWXQVJ

Turn 3: Top paragraphs indices: [29, 12, 26]
Generate answer summary (norm): 1.5655 with Answer: 0W6VIK

Turn 4: Top paragraphs indices: [17, 7, 0]
Generate answer summary (norm): 1.5160 with Answer: A2REWG

Turn 5: Top paragraphs indices: [17, 22, 21]
Generate answer summary (norm): 1.1919 with Answer: OGPFHB

